In [1]:
import requests
import gzip
import json
import csv

# 1. APIキーの設定
API_KEY = '64d1c7a6a4814486a88a4279f2be3022'  # ここにあなたのAPIキーを入力してください

In [ ]:

# 2. APIパラメータの設定
params = {
    'priceClassification': '01',  # 価格情報区分コード
    'year': '2023',               # 最新の取引時期（年）
    'area': '13',                 # 都道府県コード（東京都）
    'language': 'ja'              # 言語（'ja'は日本語）
}

# 3. リクエストヘッダーの設定
headers = {
    'Ocp-Apim-Subscription-Key': API_KEY,
    'Accept-Encoding': 'gzip'
}

# 4. APIリクエストURLの構築
base_url = 'https://www.reinfolib.mlit.go.jp/ex-api/external/XIT001'
response = requests.get(base_url, params=params, headers=headers)

# リクエストURLを出力
print(f"リクエストURL: {response.url}")

# 5. レスポンスの確認と解凍
if response.status_code == 200:
    # gzip解凍
    try:
        content = gzip.decompress(response.content)
    except OSError as e:
        print("gzipの解凍に失敗しました。")
        print(f"エラー内容: {e}")
        print("レスポンス内容をそのまま表示します。")
        print(response.content)
        exit()

    # JSONデータをパース
    try:
        data = json.loads(content.decode('utf-8'))
    except json.JSONDecodeError as e:
        print("JSONのパースに失敗しました。")
        print(f"エラー内容: {e}")
        print("解凍後のコンテンツを表示します。")
        print(content.decode('utf-8', errors='replace'))
        exit()

    # データの抽出
    transactions = data

    if transactions:
        # CSVファイルへの書き込み
        with open('real_estate_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
            # CSVの列名を定義
            fieldnames = [
                'Type',
                'Region',
                'Prefecture',
                'Municipality',
                'DistrictName',
                'TradePrice',
                'PricePerUnit',
                'Area',
                'UnitPrice',
                'BuildingYear',
                'Structure',
                'Use',
                'Purpose',
                'Period',
                'Remarks'
            ]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()

            # データ行を書き込み
            for transaction in transactions:
                data_row = {field: transaction.get(field, '') for field in fieldnames}
                writer.writerow(data_row)
        print("データの取得とCSVへの書き込みが完了しました。")
    else:
        print("指定した条件でデータが見つかりませんでした。")
else:
    print(f"エラーが発生しました。ステータスコード: {response.status_code}")
    print(f"メッセージ: {response.text}")
    print(f"レスポンスヘッダー: {response.headers}")
